## Burger's equation

Tools: jax (to install `pip install jax`)
Goal: have a first simple 1D model to work with.

https://arxiv.org/pdf/1711.10561.pdf

Burger's equation becomes:
$$
u_t + u \times u_x − (0.01/π)u_{xx} = 0, x ∈ [−1, 1], t ∈ [0, 1], \\
u(0, x) = − sin(πx), \\
u(t, −1) = u(t, 1) = 0
$$

In [ ]:
import jax.numpy as np
from jax import grad, jit, vmap, jacfwd, jacrev
from jax import random

# Jax uses a state based random number generation 
# process, which is much less error prone than (hidden) 
# stateless cases in tensorflow, pytorch, but more 
# cumbersome. You need to split the key into 2 parts 
# and use subparts to generate your random numbers.
key = random.PRNGKey(0)
key, subkey = random.split(key)

W = random.normal(subkey, (2, 1))
b = np.zeros(1)
params = (W, b)

# Simple u function: only 3 params!
def u(t, x, params):
    W, b = params
    xandt = np.concatenate((t, x))
    return np.dot(xandt, W) + b

In [ ]:
# t = 0 border condition
def u0(x):
    return - np.sin(np.pi * x)

# u_xx
def hessian(f, index_derivation=0):
    return jacfwd(jacrev(f,index_derivation),index_derivation)

# function f could be defined with grad instead of jacobians
# because we only have scalars here.
def f(t_, x_, params_):
    u_out = u(t_, x_, params_)
    u_t = jacfwd(u,0)(t_, x_, params_)
    u_x = jacfwd(u,1)(t_, x_, params_)
    u_xx = hessian(u,1)(t_, x_, params_)
    f_out = u_t + u_out*u_x - (0.01/np.pi)*u_xx
    return f_out

In [ ]:
# A test point
x_test = np.ones(1) * 0.25
t_test = np.ones(1) * 0.25

u(t_test, x_test, params), f(t_test, x_test, params)

In [ ]:
def mse_f(t_, x_, params_):
    return np.mean(f(t_, x_, params_)**2)

mse_f(t_test, x_test, params)

In [ ]:
def mse_u(t_, x_, u_, params_):
    return np.mean((u_ - u(t_, x_, params_))**2)

mse_u(np.ones(1), np.zeros(1), np.zeros(1), params)

In [ ]:
def mse(t_, x_, u_, params_):
    return mse_u(t_, x_, u_, params_) + mse_f(t_, x_, params_)

In [ ]:
def gradient_params(t_, x_, u_, params_):
    return grad(mse, 3)(t_, x_, u_, params_)

# Will output 2 objects: gradients wrt W and b
gradient_params(np.ones(1), np.zeros(1), np.zeros(1), params)

#### Data and learning

We build $N_u = 100$ boundary data points as mentionned in the paper. Half of them for $t=0$, the other half for $x= \pm 1$

In [ ]:
N_u = 100
data = []

for i in range(N_u):
    x_data,t_data,u_data = 0.0, 0.0, 0.0
    key, subkey = random.split(key)
    if  random.uniform(subkey)>0.5:
        t_data = 0
        key, subkey = random.split(key)
        x_data = random.uniform(subkey)*2-1
        u_data = u0(x_data)
        # t=0, u= -sin(pi x)
    else:
        key, subkey = random.split(key)
        t_data = random.uniform(subkey)
        key, subkey = random.split(key)
        x_data = (random.uniform(subkey)>0.5)*2-1
        u_data = 0
    data.append([x_data,t_data,u_data])
data

More Numpy way to build data (no loops)

In [ ]:
key, subkey = random.split(key)
data_type = random.uniform(key, (100,))>0.5
key, subkey1, subkey2, subkey3 = random.split(key, 4)
x_data = data_type * (random.uniform(subkey1, (100,))*2.0-1.0)+ \
    (1-data_type) * ((random.uniform(subkey2, (100,))>0.5)*2.0-1.0)
t_data = data_type * 0 + ((1-data_type) * random.uniform(subkey3, (100,)))
u_data = data_type * u0(x_data)

x_data, t_data, u_data

todo: 
- sample $N_f$ points for evaluation of $f$ 
- build a real neural network
- training loop
- display results

## Modèle KPP

$$
\begin{equation} \label{eq:KPP_homog}
  \partial_t u(t,x) = D \Delta u + r u (1 - u), \ t>0, \ x\in \Omega \subset \mathbb{R}^2,
\end{equation}
$$


avec la condition initiale $u(0,\cdot)=u_0(\cdot)$ dans $\Omega$ et la condition au bord $u(t,\cdot )=0$ sur $\partial\Omega$ pour tout $t>0$. On pourra prendre $\Omega=(0,1)\times(0,1)$.

To be continued.